# Data processing for model

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

#import matplotlib.pyplot as plt
#import seaborn as sns

#from sklearn.preprocessing import MinMaxScaler
#import geopandas as gpd

## Read in and process dependent variable - turnover

Read turnover data from csv into a dataframe.

In [2]:
# annual and monthly data
#annual_url = '../01_data/processed_annual_turnover.csv'
annual_url = '../01_data/annual_turnover.csv'

# monthly_url = '../01_data/processed_monthly_turnover.csv'

# staff group ref table
ref_sg = pd.read_csv('../01_data/ref_sg_grouped.csv')

annual_df = pd.read_csv(annual_url, parse_dates=['month_year'])
annual_df = annual_df.drop(['n'],axis=1)

annual_df.rename(columns={'ORG_CODE':'org_code',
                        'STAFF_GROUP':'staff_group'}, inplace=True)

# drop unneeded columns
annual_df = annual_df.drop(['ORG_TYPE','join_HC','leave_HC','denom_HC','denom_FTE_12',
               'denom_HC_12','denom_HC_mean'],axis=1)

#KeyError: "['org_type', 'denom_FTE_start', 'denom_HC_start', 'denom_HC_average'] not found in axis"

# group staff groups with mapping table
annual_df = pd.merge(annual_df, ref_sg, on='staff_group', how='left')

annual_df.info()

# monthly_df = pd.read_csv(monthly_url, parse_dates=['month_year'])
# monthly_df = monthly_df.drop(['n'],axis=1)
# monthly_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201750 entries, 0 to 201749
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   org_code        201750 non-null  object        
 1   staff_group     201750 non-null  object        
 2   month_year      201750 non-null  datetime64[ns]
 3   join_FTE        155609 non-null  float64       
 4   leave_FTE       157764 non-null  float64       
 5   denom_FTE       197137 non-null  float64       
 6   region_name     201750 non-null  object        
 7   denom_FTE_mean  201447 non-null  float64       
 8   grouped_sg      201750 non-null  object        
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 13.9+ MB


clean and process DV dataframes

In [3]:
# add leaver and joiner rates column
annual_df['leaver_rate'] = annual_df['leave_FTE']/annual_df['denom_FTE_mean']
annual_df['joiner_rate'] = annual_df['join_FTE']/annual_df['denom_FTE_mean']

In [4]:
annual_df1 = annual_df.copy()

annual_df1.head()

,org_code,staff_group,month_year,join_FTE,leave_FTE,denom_FTE,region_name,denom_FTE_mean,grouped_sg,leaver_rate,joiner_rate
0,TAJ,All staff groups,2023-07-01,614.16006,346.48324,3453.54378,MIDLANDS,3324.193505,all,0.104231,0.184755
1,TAJ,Central functions,2023-07-01,42.25333,35.56669,278.88667,MIDLANDS,273.472425,non-clinical,0.130056,0.154507
2,TAJ,HCHS doctors (exc. junior Drs),2023-07-01,4.80000,5.60000,122.66591,MIDLANDS,120.940910,clinical,0.046304,0.039689
3,TAJ,"Hotel, property & estates",2023-07-01,22.14668,15.36000,181.20004,MIDLANDS,180.306715,non-clinical,0.085188,0.122828
4,TAJ,Managers,2023-07-01,21.80000,11.05333,93.59334,MIDLANDS,78.290005,non-clinical,0.141184,0.278452


In [5]:
# drop 'all staff' group
annual_df1 = annual_df1[annual_df1['grouped_sg'] != 'all']

In [6]:
# Assuming 'df' is your dataframe
# Group by 'org', 'date', and 'staff_group' and sum the 'denom_FTE'
grouped_df = annual_df1.groupby(['org_code', 'month_year', 'grouped_sg'])['denom_FTE'].sum().reset_index()

# Calculate the total 'denom_FTE' for each 'org' and 'date'
total_denom_FTE = grouped_df.groupby(['org_code', 'month_year'])['denom_FTE'].transform('sum')

# Create new columns for the proportion of each staff group within each organization and date
grouped_df['prop_staff_group'] = (grouped_df['denom_FTE'] / total_denom_FTE)

# Pivot the dataframe to have columns for each staff group
pivot_df = grouped_df.pivot_table(index=['org_code', 'month_year'], columns='grouped_sg', values='prop_staff_group', 
                                  fill_value=0).reset_index()

# Rename the columns to include the prefix '%_'
#pivot_df.columns = [f'%_{col}_staff' if col != ('org', 'date') else col for col in pivot_df.columns]

# Merge the pivot table back to the original dataframe
annual_df2 = pd.merge(annual_df1, pivot_df, on=['org_code', 'month_year'], how='left')

# If there are missing values, you may want to fill them with 0
annual_df2.fillna(0, inplace=True)

In [7]:
annual_df2.head()

,org_code,staff_group,month_year,join_FTE,leave_FTE,denom_FTE,region_name,denom_FTE_mean,grouped_sg,leaver_rate,joiner_rate,clinical,non-clinical
0,TAJ,Central functions,2023-07-01,42.25333,35.56669,278.88667,MIDLANDS,273.472425,non-clinical,0.130056,0.154507,0.577655,0.422345
1,TAJ,HCHS doctors (exc. junior Drs),2023-07-01,4.80000,5.60000,122.66591,MIDLANDS,120.940910,clinical,0.046304,0.039689,0.577655,0.422345
2,TAJ,"Hotel, property & estates",2023-07-01,22.14668,15.36000,181.20004,MIDLANDS,180.306715,non-clinical,0.085188,0.122828,0.577655,0.422345
3,TAJ,Managers,2023-07-01,21.80000,11.05333,93.59334,MIDLANDS,78.290005,non-clinical,0.141184,0.278452,0.577655,0.422345
4,TAJ,Nurses & health visitors,2023-07-01,155.07630,108.61440,1007.00718,MIDLANDS,986.729345,clinical,0.110075,0.157162,0.577655,0.422345


In [8]:
## Calculate a regional sip FTE by staff group column for later use
# staff in post by staff group for each region by month
df_sip_region = annual_df2.groupby(['month_year','staff_group','region_name'])['denom_FTE'].sum().reset_index()
# # # merge the df_sip_region DataFrame  
annual_df3 = pd.merge(annual_df2, df_sip_region, on=['month_year','region_name','staff_group'], suffixes=('', '_region'))

In [9]:
annual_df3.rename(columns={'denom_FTE_total': 'total_sip_FTE','denom_FTE_region':'sip_FTE_region'}, inplace=True)
annual_df3.head()

,org_code,staff_group,month_year,join_FTE,leave_FTE,denom_FTE,region_name,denom_FTE_mean,grouped_sg,leaver_rate,joiner_rate,clinical,non-clinical,sip_FTE_region
0,TAJ,Central functions,2023-07-01,42.25333,35.56669,278.88667,MIDLANDS,273.472425,non-clinical,0.130056,0.154507,0.577655,0.422345,17478.85472
1,RYW,Central functions,2023-07-01,60.07332,63.52000,339.09167,MIDLANDS,483.531500,non-clinical,0.131367,0.124239,0.599197,0.400803,17478.85472
2,RYK,Central functions,2023-07-01,6.00000,5.00000,22.70667,MIDLANDS,20.586670,non-clinical,0.242876,0.291451,0.509536,0.490464,17478.85472
3,RYG,Central functions,2023-07-01,42.77333,40.58667,228.20186,MIDLANDS,233.115930,non-clinical,0.174105,0.183485,0.601556,0.398444,17478.85472
4,RYA,Central functions,2023-07-01,21.30000,31.54667,229.50426,MIDLANDS,222.146660,non-clinical,0.142008,0.095883,0.904498,0.095502,17478.85472


In [10]:
# replace inf values with nan (can happen with rate calcs)
# df4.replace([np.inf, -np.inf], np.nan, inplace=True)
# # transform nans to zeros
# df4.fillna(0, inplace=True)

# Add a small constant to avoid taking the log of zero
# small_constant = 1e-5
    
# # log scale the total_SIP_FTE column to be in line with other variables. proxy for size of organisation
# annual_df3['log_total_sip_FTE'] = np.log(annual_df3['total_sip_FTE'] + small_constant)

# # drop unused columns (keep total SIP FTE for calculating vacancy rates later)
# annual_df3.drop(['join_FTE','leave_FTE','denom_FTE','total_sip_FTE'], axis=1, inplace=True)


In [11]:
annual_df3.head()

,org_code,staff_group,month_year,join_FTE,leave_FTE,denom_FTE,region_name,denom_FTE_mean,grouped_sg,leaver_rate,joiner_rate,clinical,non-clinical,sip_FTE_region
0,TAJ,Central functions,2023-07-01,42.25333,35.56669,278.88667,MIDLANDS,273.472425,non-clinical,0.130056,0.154507,0.577655,0.422345,17478.85472
1,RYW,Central functions,2023-07-01,60.07332,63.52000,339.09167,MIDLANDS,483.531500,non-clinical,0.131367,0.124239,0.599197,0.400803,17478.85472
2,RYK,Central functions,2023-07-01,6.00000,5.00000,22.70667,MIDLANDS,20.586670,non-clinical,0.242876,0.291451,0.509536,0.490464,17478.85472
3,RYG,Central functions,2023-07-01,42.77333,40.58667,228.20186,MIDLANDS,233.115930,non-clinical,0.174105,0.183485,0.601556,0.398444,17478.85472
4,RYA,Central functions,2023-07-01,21.30000,31.54667,229.50426,MIDLANDS,222.146660,non-clinical,0.142008,0.095883,0.904498,0.095502,17478.85472


The data show the full time equivalent (FTE) number of leavers by organisation and staff group for the previous 12-month period from the date. It also shows the number of staff in post (SIP) FTE averaged over the 12-month period to date.

In [12]:
sorted(annual_df1['staff_group'].unique())

['Ambulance staff',
 'Central functions',
 'HCHS doctors (exc. junior Drs)',
 'Hotel, property & estates',
 'Managers',
 'Midwives',
 'Nurses & health visitors',
 'Other staff or those with unknown classification',
 'Scientific, therapeutic & technical staff',
 'Senior managers',
 'Support to ST&T staff',
 'Support to ambulance staff',
 'Support to doctors, nurses & midwives']

## Load independent variable 1 - local unemployment

Load data about local unemployment so we can use it as a regressor

In [13]:
url_r1 = '../01_data/ONS_localunemployment_monthly.csv'
df_r1 = pd.read_csv(url_r1, parse_dates=['Date'])

df_r1.rename(columns={'%':'local_unemployment','Date':'month_year',
                      'NHSE region name':'region_name'},inplace=True)
df_r1 = df_r1.sort_values('month_year')
df_r1.tail()

,month_year,region_name,local_unemployment
2619,2023-06-01,London,0.0480
2620,2023-06-01,Midlands,0.0455
2621,2023-06-01,North East,0.0520
2622,2023-06-01,North West,0.0530
2624,2023-06-01,South West,0.0360


## Load IV 2 - sickness absence

Load data about sickness absence to use as second regressor

In [37]:
url_r2 = '../01_data/sickness_benchmarking.csv'
df_r2 = pd.read_csv(url_r2, parse_dates=['DATE'])
trust_types_todrop = ['Clinical Commissioning Group','Integrated Care Board']
df_r2 = df_r2[~df_r2['CLUSTER_GROUP'].isin(trust_types_todrop)]
df_r2 = df_r2.drop(['BENCHMARK_GROUP','ORG_NAME',
                     'NHSE_REGION_CODE','CLUSTER_GROUP'],axis=1)
df_r2.rename(columns={'ORG_CODE':'org_code','DATE':'month_year',
                       'NHSE_REGION_NAME':'region_name','STAFF_GROUP':'staff_group',
                       'FTE_DAYS_LOST':'fte_days_lost','FTE_DAYS_AVAILABLE':'fte_days_available'},inplace=True)
merge_cols = ['month_year', 'org_code','region_name','staff_group']
df_r2['sickness_absence'] = df_r2['fte_days_lost']/df_r2['fte_days_available']
df_r2 = df_r2.reset_index(drop=True)
df_r2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192606 entries, 0 to 192605
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   month_year          192606 non-null  datetime64[ns]
 1   staff_group         192606 non-null  object        
 2   file_date           192606 non-null  object        
 3   org_code            192606 non-null  object        
 4   region_name         192606 non-null  object        
 5   fte_days_lost       166612 non-null  float64       
 6   fte_days_available  166612 non-null  float64       
 7   sickness_absence    166358 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 11.8+ MB


Create 12-month rolling sickness absence column for use with annual turnover data

In [38]:
df_r2['month_year'] = pd.to_datetime(df_r2['month_year'])

# Sort the DataFrame by organisation, staff_group, and month
df_r2.sort_values(by=['month_year'], inplace=True)

# # Calculate the rolling sums for days lost and days available
# df_r2['rolling_days_lost'] = df_r2.groupby(['org_code', 
#                         'staff_group'])['fte_days_lost'].rolling(window=12, min_periods=1).sum().reset_index(level=[0, 1], drop=True)

# df_r2['rolling_days_available'] = df_r2.groupby(['org_code', 
#                         'staff_group'])['fte_days_available'].rolling(window=12, min_periods=1).sum().reset_index(level=[0, 1], drop=True)

# # Calculate the rolling sickness absence rate
# df_r2['annual_sickness_absence'] = df_r2['rolling_days_lost'] / df_r2['rolling_days_available']

# drop fte_days_lost fte_days_available, rolling_days_available and rolling_days_lost columns
df_r2.drop(columns=['fte_days_lost', 'fte_days_available','file_date','region_name'], inplace=True)
#, 'rolling_days_available', 'rolling_days_lost'], inplace=True)

df_r2.head()

,month_year,staff_group,org_code,sickness_absence
192605,2018-04-01,Other staff or those with unknown classification,8JX76,NaN
167398,2018-04-01,Nurses & health visitors,RAT,0.051255
167399,2018-04-01,Other staff or those with unknown classification,RAT,NaN
167400,2018-04-01,"Scientific, therapeutic & technical staff",RAT,0.022210
167401,2018-04-01,Senior managers,RAT,0.010011


## Load IV 3 - reasons for sickness absence

Add data about reasons for sickness absence

In [16]:
url_r3 = '../01_data/sickness_absence_reason_pivot.csv'
df_r3 = pd.read_csv(url_r3, parse_dates=['Date'])
#df_r3 = df_r3.drop(['FTE days lost'],axis=1)
df_r3.rename(columns={'Date':'month_year','Staff group':'staff_group'},inplace=True)
#df_r2 = df_r2.reset_index(drop=True)

# drop least frequent reasons for absence
df_r3 = df_r3.drop(['substance_abus','asthma',
                    'dental','blood_disorder','endocrine',
                    'eye','skin_disorders','nervous_system',
                    'gynaecological','unknown','pregnancy_related',
                    'other'],axis=1)

# Replace NaN values with 0 
df_r3 = df_r3.fillna(0)
df_r3.info()
# national level data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1413 entries, 0 to 1412
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   month_year      1413 non-null   datetime64[ns]
 1   staff_group     1413 non-null   object        
 2   anxiety_stress  1413 non-null   float64       
 3   back_problems   1413 non-null   float64       
 4   gastro          1413 non-null   float64       
 5   headache_mig    1413 non-null   float64       
 6   infectious_dis  1413 non-null   float64       
 7   other_msk       1413 non-null   float64       
 8   respiratory     1413 non-null   float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 99.5+ KB


## Load IV 4 and 5 - staff vacancies

In [17]:
url_sg_ref = '../01_data/ref_sg_vacancy.csv'
df_sg_ref = pd.read_csv(url_sg_ref)
df_sg_ref.head()

,staff_group,vacancy_sg,all
0,Ambulance staff,Allied Health Professionals,All staff groups
1,Central functions,Administrative and Clerical,All staff groups
2,HCHS doctors (exc. junior Drs),Medical and Dental,All staff groups
3,"Hotel, property & estates",Estates and Ancillary,All staff groups
4,Managers,Administrative and Clerical,All staff groups


In [18]:
url_r4 = '../01_data/vacancy_ESR.csv'
df_r4 = pd.read_csv(url_r4,parse_dates=['month_year'],dayfirst=True)

df_r4 = df_r4.drop(['Published month','Published quarter','England'],axis=1)
df_r4.rename(columns={'NWD Staff Group':'vacancy_sg','NHS England region':'region_name',
                        'Vacancy Wte':'vacancy_FTE'},inplace=True)

df_r4 = df_r4.fillna(0)

# Remove code in brackets
df_r4['region_name'] = df_r4['region_name'].str[:-6].str.rstrip()

# Add staff groupings to match other datasets
df_r4 = pd.merge(df_r4, df_sg_ref, on='vacancy_sg',how='left')

df_r4 = df_r4.drop(['all'],axis=1)

df_r4.info()

# regional level

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8387 entries, 0 to 8386
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   month_year   8387 non-null   datetime64[ns]
 1   vacancy_sg   8387 non-null   object        
 2   region_name  8387 non-null   object        
 3   vacancy_FTE  8387 non-null   float64       
 4   staff_group  7948 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 327.7+ KB


In [19]:
url_r5 = '../01_data/vacancy_TRAC.csv'
df_r5 = pd.read_csv(url_r5,parse_dates=['month_year'],dayfirst=True)

df_r5 = df_r5.drop(['Published month','Published quarter','England'],axis=1)

df_r5.rename(columns={'NWD Staff Group':'vacancy_sg','NHS England region':'region_name',
                        'Advertised FTE':'advertised_FTE'},inplace=True)

df_r5 = df_r5.fillna(0)

# Remove region code in brackets
df_r5['region_name'] = df_r5['region_name'].str[:-6].str.rstrip()

# Add staff groupings to match other datasets
df_r5 = pd.merge(df_r5, df_sg_ref, on='vacancy_sg',how='left')

df_r5 = df_r5.drop(['all'],axis=1)

# regional level

df_r5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4226 entries, 0 to 4225
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   month_year      4226 non-null   datetime64[ns]
 1   vacancy_sg      4226 non-null   object        
 2   region_name     4226 non-null   object        
 3   advertised_FTE  4226 non-null   float64       
 4   staff_group     3999 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 165.2+ KB


## Load IV 6 - Reasons for leaving

In [20]:
url_r6 = '../01_data/rfl_jun23.csv'
df_r6_a = pd.read_csv(url_r6,parse_dates=['month_year'],dayfirst=True)


In [21]:
df_r6_a.head()

,quarter,month_year,rfl_death_in_service,rfl_dismissal,rfl_end_of_ft,rfl_flexibility,rfl_health,rfl_incompat_relations,rfl_other,rfl_pay_reward,rfl_pregnancy,rfl_progression_cpd,rfl_relocation,rfl_retirement,rfl_unknown,rfl_work_life_balance,rfl_workforce_transform,rfl_neg_RFL
0,Q1-2016-17,2016-04-01,0.01,0.03,0.09,0.02,0.02,0.01,0.0,0.11,0,0.03,0.12,0.16,0.21,0.10,0.09,0.33
1,Q1-2016-17,2016-05-01,0.01,0.03,0.09,0.02,0.02,0.01,0.0,0.11,0,0.03,0.12,0.16,0.21,0.10,0.09,0.33
2,Q1-2016-17,2016-06-01,0.01,0.03,0.09,0.02,0.02,0.01,0.0,0.11,0,0.03,0.12,0.16,0.21,0.10,0.09,0.33
3,Q2-2016-17,2016-07-01,0.00,0.02,0.34,0.01,0.01,0.01,0.0,0.08,0,0.05,0.09,0.10,0.15,0.06,0.08,0.24
4,Q2-2016-17,2016-08-01,0.00,0.02,0.34,0.01,0.01,0.01,0.0,0.08,0,0.05,0.09,0.10,0.15,0.06,0.08,0.24


In [22]:
# to_keep = ['month_year','rfl_dismissal','rfl_end_of_ft',
#             'rfl_flexibility','rfl_health','rfl_pay_reward','rfl_progression_cpd',
#             'rfl_relocation','rfl_retirement','rfl_work_life_balance','rfl_workforce_transform']
to_keep = ['month_year','rfl_neg_RFL']
df_r6 = df_r6_a[to_keep]
#df_r6 = df_r6_a
df_r6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   month_year   87 non-null     datetime64[ns]
 1   rfl_neg_RFL  87 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.5 KB


## Merge IV dfs to main df

In [39]:
annual_df3.head()

,org_code,staff_group,month_year,join_FTE,leave_FTE,denom_FTE,region_name,denom_FTE_mean,grouped_sg,leaver_rate,joiner_rate,clinical,non-clinical,sip_FTE_region
0,TAJ,Central functions,2023-07-01,42.25333,35.56669,278.88667,MIDLANDS,273.472425,non-clinical,0.130056,0.154507,0.577655,0.422345,17478.85472
1,RYW,Central functions,2023-07-01,60.07332,63.52000,339.09167,MIDLANDS,483.531500,non-clinical,0.131367,0.124239,0.599197,0.400803,17478.85472
2,RYK,Central functions,2023-07-01,6.00000,5.00000,22.70667,MIDLANDS,20.586670,non-clinical,0.242876,0.291451,0.509536,0.490464,17478.85472
3,RYG,Central functions,2023-07-01,42.77333,40.58667,228.20186,MIDLANDS,233.115930,non-clinical,0.174105,0.183485,0.601556,0.398444,17478.85472
4,RYA,Central functions,2023-07-01,21.30000,31.54667,229.50426,MIDLANDS,222.146660,non-clinical,0.142008,0.095883,0.904498,0.095502,17478.85472


In [24]:
sorted(annual_df3['staff_group'].unique())

['Ambulance staff',
 'Central functions',
 'HCHS doctors (exc. junior Drs)',
 'Hotel, property & estates',
 'Managers',
 'Midwives',
 'Nurses & health visitors',
 'Other staff or those with unknown classification',
 'Scientific, therapeutic & technical staff',
 'Senior managers',
 'Support to ST&T staff',
 'Support to ambulance staff',
 'Support to doctors, nurses & midwives']

In [25]:
# # ref table with org information
# url_ref_org = '../01_data/REF_ORGANISATION.csv'
# ref_org = pd.read_csv(url_ref_org)

# ref_org = ref_org.drop(['Org_Open_Date','Org_Close_Date', 'Org_Name','Org_Type',
#                         'Org_Code_For_Use','Org_Name_For_Use','Org_Status',
#                         'Legacy_Org_Close_Date','UDALFileID','Org_System_Name',
#                         ' NHS Provider flag ',' Total WTE recorded '],axis=1)

# ref_org.rename(columns={'Org_Code_For_Join':'org_code','Org_Type_Grouped':'org_type'},inplace=True)

# ref_org.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21813 entries, 0 to 21812
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   org_code         21813 non-null  object
 1   Org_Region_Code  21647 non-null  object
 2   Org_Region_Name  21647 non-null  object
 3   Org_System_Code  21493 non-null  object
 4   Org_ICB_Name     21493 non-null  object
 5   org_type         21813 non-null  object
 6   Org_Post_Code    21813 non-null  object
dtypes: object(7)
memory usage: 1.2+ MB


In [26]:
#def merge_ivs(df, ref_org, df_r1, df_r2, df_r3,df_r4,df_r5,df_r6):
    # org reference data
#df1 = pd.merge(annual_df3, ref_org, on=['org_code'],how='left')

In [40]:
# local unemployment rate
df1 = pd.merge(annual_df3, df_r1, on=['month_year', 'region_name'],how='left')
df1 = df1.sort_values('month_year')

In [41]:
# sickness absence
r2_merge_cols = ['month_year', 'org_code','staff_group']
df2 = pd.merge(df1, df_r2, on=r2_merge_cols,how='left')

In [42]:
# reason for sickness absence
r3_merge_cols = ['month_year','staff_group']
df3 = pd.merge(df2, df_r3, on=r3_merge_cols,how='left')

In [43]:
df3.head()

,org_code,staff_group,month_year,join_FTE,leave_FTE,denom_FTE,region_name,denom_FTE_mean,grouped_sg,leaver_rate,...,sip_FTE_region,local_unemployment,sickness_absence,anxiety_stress,back_problems,gastro,headache_mig,infectious_dis,other_msk,respiratory
0,RA2,"Scientific, therapeutic & technical staff",2017-04-01,0.0,0.0,580.91610,SOUTH EAST,580.91610,non-clinical,0.0,...,18636.12134,NaN,NaN,0.286571,0.060627,0.119881,0.029726,0.008406,0.119681,0.033556
1,RN3,Central functions,2017-04-01,0.0,0.0,319.65736,SOUTH WEST,319.65736,non-clinical,0.0,...,6776.75914,NaN,NaN,0.336194,0.059593,0.099724,0.027558,0.009475,0.095223,0.046109
2,RK9,Central functions,2017-04-01,0.0,0.0,647.55013,SOUTH WEST,647.55013,non-clinical,0.0,...,6776.75914,NaN,NaN,0.336194,0.059593,0.099724,0.027558,0.009475,0.095223,0.046109
3,RJ8,Central functions,2017-04-01,0.0,0.0,150.63336,SOUTH WEST,150.63336,non-clinical,0.0,...,6776.75914,NaN,NaN,0.336194,0.059593,0.099724,0.027558,0.009475,0.095223,0.046109
4,RH8,Central functions,2017-04-01,0.0,0.0,442.96079,SOUTH WEST,442.96079,non-clinical,0.0,...,6776.75914,NaN,NaN,0.336194,0.059593,0.099724,0.027558,0.009475,0.095223,0.046109


In [44]:
# vacancy - need to calculate rate at regional level here
# use sip_FTE_region calculated earlier
r4_merge_cols = ['month_year','region_name','staff_group']

df4 = pd.merge(df3, df_r4, on=r4_merge_cols,how='left')

df4.drop(columns=['vacancy_sg'], inplace=True)

df4['vacancy_rate'] = df4['vacancy_FTE'] / df4['sip_FTE_region']


In [50]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189879 entries, 0 to 189878
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   org_code            189879 non-null  object        
 1   staff_group         189879 non-null  object        
 2   month_year          189879 non-null  datetime64[ns]
 3   join_FTE            189879 non-null  float64       
 4   leave_FTE           189879 non-null  float64       
 5   denom_FTE           189879 non-null  float64       
 6   region_name         189879 non-null  object        
 7   denom_FTE_mean      189879 non-null  float64       
 8   grouped_sg          189879 non-null  object        
 9   leaver_rate         189879 non-null  float64       
 10  joiner_rate         189879 non-null  float64       
 11  clinical            189879 non-null  float64       
 12  non-clinical        189879 non-null  float64       
 13  local_unemployment  0 non-nul

In [47]:
df5 = pd.merge(df4, df_r5, on=r4_merge_cols,how='left')

df5.drop(columns=['vacancy_sg'], inplace=True)

df5['advertised_rate'] = df5['advertised_FTE'] / df5['sip_FTE_region']

df5.drop(columns=['sip_FTE_region','advertised_FTE','vacancy_FTE'], inplace=True)


In [51]:
# reasons for leaving
df6 = pd.merge(df5, df_r6, on='month_year',how='left')

# add region as dummy variable
df6 = pd.get_dummies(df6, columns=['region_name'], drop_first=True)

# add org type as dummy variable
#df6 = pd.get_dummies(df6, columns=['org_type'], drop_first=True)

# Extract year and month
df6['year'] = df6['month_year'].dt.year
df6['month'] = df6['month_year'].dt.month
df6 = pd.get_dummies(df6, columns=['year', 'month'], prefix=['year', 'month'])

# Convert True/False dummy variable categories to integer 0/1
bool_columns = df6.select_dtypes(include='bool').columns
df6[bool_columns] = df6[bool_columns].astype(int)

# transform nans into 0s
df6 = df6.fillna(0)

In [52]:
# Need to cut dataframe to earliest and latest data available for all fields. 
# Do this by cutting rows where all values for key variables are zero
df6 = df6[~((df6['leaver_rate'] == 0) | (df6['joiner_rate'] == 0) | (df6['sickness_absence'] == 0))]


In [53]:
df7 = df6[~((df6['rfl_neg_RFL'] == 0))]

# drop duplicates
df7.drop_duplicates(inplace=True)
    
df7.info()

<class 'pandas.core.frame.DataFrame'>
Index: 123097 entries, 29837 to 187566
Data columns (total 49 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   org_code                              123097 non-null  object        
 1   staff_group                           123097 non-null  object        
 2   month_year                            123097 non-null  datetime64[ns]
 3   join_FTE                              123097 non-null  float64       
 4   leave_FTE                             123097 non-null  float64       
 5   denom_FTE                             123097 non-null  float64       
 6   denom_FTE_mean                        123097 non-null  float64       
 7   grouped_sg                            123097 non-null  object        
 8   leaver_rate                           123097 non-null  float64       
 9   joiner_rate                           123097 non-null  float

C:\Users\MarieRogers\AppData\Local\Temp\ipykernel_17048\1636315098.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.drop_duplicates(inplace=True)


In [ ]:
#df2.drop_duplicates(subset=r2_merge_cols)


In [ ]:
#annual_df_ivs = merge_ivs(annual_df3,ref_org,df_r1, df_r2, df_r3,df_r4,df_r5,df_r6)
#monthly_df_ivs = merge_ivs(monthly_df1,df_r1, df_r2, df_r3,df_r4,df_r5,df_r6)

In [54]:
# create copy of df7 and name annual_df_ivs
annual_df_ivs = df7.copy()

In [55]:
annual_df_ivs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 123097 entries, 29837 to 187566
Data columns (total 49 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   org_code                              123097 non-null  object        
 1   staff_group                           123097 non-null  object        
 2   month_year                            123097 non-null  datetime64[ns]
 3   join_FTE                              123097 non-null  float64       
 4   leave_FTE                             123097 non-null  float64       
 5   denom_FTE                             123097 non-null  float64       
 6   denom_FTE_mean                        123097 non-null  float64       
 7   grouped_sg                            123097 non-null  object        
 8   leaver_rate                           123097 non-null  float64       
 9   joiner_rate                           123097 non-null  float

In [56]:
annual_df_ivs.to_csv(f'annual_modelling_data.csv', index=False)

In [ ]:
#monthly_df_ivs.to_csv(f'monthly_modelling_data.csv', index=False)

## OLS multiple regression

### Annual data

Specify the dependent variable (dv). All other fields to be dropped. 

In [57]:
dv = 'leaver_rate'
to_drop = ['join_FTE','leave_FTE','month_year','denom_FTE','org_code',
           'denom_FTE_mean','staff_group','grouped_sg','year_2017',                          
           dv]


Define the design matrix (X) and the dependent variable (y)


In [58]:
annual_df_ivs.reset_index(drop = True)
X = annual_df_ivs.drop(to_drop, axis=1)
y = annual_df_ivs[dv]

y.head()


29837    0.152665
29838    0.190405
29839    0.139319
29840    0.143873
29841    0.164443
Name: leaver_rate, dtype: float64

In [59]:
# Add a constant column to the design matrix
X = sm.add_constant(X)

X.info()


<class 'pandas.core.frame.DataFrame'>
Index: 123097 entries, 29837 to 187566
Data columns (total 40 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   const                                 123097 non-null  float64
 1   joiner_rate                           123097 non-null  float64
 2   clinical                              123097 non-null  float64
 3   non-clinical                          123097 non-null  float64
 4   local_unemployment                    123097 non-null  float64
 5   sickness_absence                      123097 non-null  float64
 6   anxiety_stress                        123097 non-null  float64
 7   back_problems                         123097 non-null  float64
 8   gastro                                123097 non-null  float64
 9   headache_mig                          123097 non-null  float64
 10  infectious_dis                        123097 non-null  float64
 11  o

In [60]:
# Fit the regression model
model = sm.OLS(y, X)
results = model.fit()

# Print the regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            leaver_rate   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     510.0
Date:                Wed, 22 Nov 2023   Prob (F-statistic):               0.00
Time:                        11:52:05   Log-Likelihood:             1.7582e+05
No. Observations:              123097   AIC:                        -3.516e+05
Df Residuals:                  123063   BIC:                        -3.512e+05
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [ ]:
# Convert the regression results summary to a DataFrame
results_df = pd.read_html(results.summary().tables[1].as_html(), header=0, index_col=0)[0]

# Export the DataFrame to a CSV file
results_df.to_csv("annual_regression_results.csv")

In [ ]:
results_df.sort_values(by='coef')

### Monthly data

In [ ]:
# dv = 'leaver_rate'
# to_drop = ['month_year','org_code','staff_group','annual_sickness_absence',
#            dv]

# monthly_df_ivs.reset_index(drop = True)
# X = monthly_df_ivs.drop(to_drop, axis=1)
# y = monthly_df_ivs[dv]

# #y = y.dropna()

# y.head()


In [ ]:
# # Add a constant column to the design matrix
# X = sm.add_constant(X)

# X.tail()


In [ ]:
# Fit the regression model
model = sm.OLS(y, X)
monthly_results = model.fit()

# Print the regression results
print(monthly_results.summary())

In [ ]:
# Convert the regression results summary to a DataFrame
monthly_results_df = pd.read_html(monthly_results.summary().tables[1].as_html(), header=0, index_col=0)[0]

# Export the DataFrame to a CSV file
monthly_results_df.to_csv("monthly_regression_results.csv")